<a href="https://colab.research.google.com/github/Joanachoong/Externship/blob/main/theme_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Theme Categorization Notebook

This notebook contains the **full code and explanations** used to categorize Glassdoor reviews and YouTube transcripts into topic themes. It:

- Loads the combined CSV `glassdoor_and_youtube_sentiment.csv` (created earlier).
- Defines a detailed `theme_keywords` dictionary with multi-word phrases and keywords for each theme.
- Provides utility functions for text normalization and theme matching.
- Categorizes **Glassdoor** fields: `cleaned_summary`, `cleaned_reviewPros`, `cleaned_reviewCons` (adds per-field and combined theme columns).
- Categorizes **YouTube** field: `transcript_cleaned` (adds `categorized_transcript` and updates `categorized_combined`).
- Offers options to increase precision (require multiple keyword hits), produce one-hot columns for top themes, and save outputs.

> Notes:
- Matching is case-insensitive and prefers phrase matches (e.g., "conveyor belt") before single-word matches.
- The notebook is self-contained and runnable in a standard Python environment with `pandas` installed.


In [ ]:
# Imports
import re
from collections import Counter
import pandas as pd

# Adjust display options for nicer notebook outputs
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 200


In [ ]:
# Load the combined CSV file (make sure the path is correct)
input_path = '/mnt/data/glassdoor_and_youtube_sentiment.csv'
try:
    df = pd.read_csv(input_path)
    print('Loaded', len(df), 'rows from', input_path)
except FileNotFoundError:
    raise FileNotFoundError(f"Could not find {input_path}. Make sure the combined CSV exists.")

# Quick peek
df.head(3)

In [ ]:
# Detailed theme_keywords dictionary
# (Expanded with many phrases and variants — you can tune this list)
theme_keywords = {
    "High-Velocity Physical Strain & Injury Risk": [
        "conveyor belt", "conveyor", "belt never stops", "pace", "high pace", "quota", "athletic",
        "lift", "lifting", "heavy lifting", "bending", "standing all day", "standing on concrete",
        "walking miles", "repetitive", "repetitive motion", "back pain", "shoulder pain", "knee pain",
        "wrist pain", "body breakdown", "fatigue", "exhaustion from work", "energy drink", "redbull",
        "cramps", "physically draining", "strained", "hurt on the job", "injury", "hurt", "sprain", "strain",
        "long-term body damage", "osha", "injury rate"
    ],

    "Surveillance-Driven Micromanagement": [
        "time off task", "tot", "written up", "written warning", "tracked", "monitor", "monitoring",
        "surveil", "camera", "cameras", "algorithmic", "algorithm", "metrics", "score", "rate",
        "productivity target", "upt", "u p t", "scored", "scanning", "scanned", "badge scan",
        "tracking every second", "micro-manage", "micromanage", "micromanaged", "policing", "policed",
        "policed bathroom", "bathroom break", "scrutiny", "logged"
    ],

    "Operational Inflexibility & Burnout": [
        "mandatory overtime", "met", "10 hour", "12 hour", "10-12 hours", "overtime", "forced overtime",
        "unpredictable schedule", "schedule change", "shift swap", "point system", "attendance points",
        "attendance policy", "no flexibility", "no time for family", "work-life balance", "too tired for life",
        "burnout", "exhausted after shift", "no recovery time", "cannot take leave", "pto denied", "unpaid time off",
        "upt", "unpaid"
    ],

    "Rigid Pay Structure & Caps": [
        "step plan", "step increase", "three year cap", "pay cap", "no raises", "incremental raise",
        "stuck at", "pay freeze", "wage cap", "hourly cap", "no merit", "pay plateau", "raise denied",
        "salary cap", "compensation structure", "comp plan"
    ],

    "Temporary vs. Long-Term Career": [
        "seasonal", "temporary", "temp", "contract", "one year", "two years", "short-term", "stepping stone",
        "no career", "no future", "retention", "turnover", "quit after season"
    ],

    "PA Advancement / Promotion Bottleneck": [
        "process assistant", "pa role", "promotion to pa", "promotion criteria", "hard to get promoted",
        "promotion blocked", "no clear path", "promotion unfair", "seniority only"
    ],

    "Safety & Injury Reporting": [
        "osha", "reporting", "incident report", "near miss", "ppe", "safety training", "no ppe", "unsafe",
        "hazard", "unsafe equipment", "not reported", "cover up", "safety walk", "safety protocol",
        "slip", "trip", "fall"
    ],

    "Compensation & Benefits": [
        "pay", "wage", "salary", "starting salary", "competitive pay", "benefits", "healthcare", "medical",
        "401k", "retirement", "vacation", "paid time off", "day 1 benefits", "insurance", "bonus",
        "holiday pay", "sick pay"
    ],

    "Training Gaps": [
        "training", "onboarding", "no onboarding", "no training", "we had to train each other", "no support",
        "confused", "learn on the job", "left alone", "no mentor", "cross-training", "no assigned trainer",
        "no one assigned to us", "first day invisible", "felt invisible", "no orientation"
    ],

    "Scheduling Issues": [
        "shift", "hours", "schedule", "night shift", "weekend", "shift swap", "prefered schedule denied",
        "schedule posted late", "short notice", "last minute", "unpredictable", "rota", "roster", "on-call",
        "shift cancellation"
    ],

    "Work Conditions (Physical)": [
        "cold warehouse", "hot floor", "no ventilation", "no chairs", "concrete floors", "wet floor", "dirty",
        "smelly", "lighting", "noise", "broken equipment", "standing", "no break area", "broken restroom",
        "bathroom conditions"
    ],

    "Emotional Tone & Voice": [
        "felt invisible", "i felt", "i was", "angry", "frustrated", "upset", "happy", "grateful", "appreciated",
        "proud", "demoralized", "dehumanized", "disrespected", "scared", "afraid", "unsafe", "stress",
        "stressed", "anxious", "depressed"
    ],

    "Management Support & Communication": [
        "manager", "supervisor", "no support", "ignored", "only interacts to enforce", "never heard from manager",
        "lack of leadership", "feedback never given", "no recognition", "sends emails only", "communication breakdown",
        "questions ignored", "escalation ignored"
    ],

    "Praise & Positive Signals": [
        "good pay", "solid option", "clean working environment", "day 1 benefits", "helpful teammates",
        "teamwork", "learned a lot", "fast promotion", "flexible manager", "great benefits", "positive",
        "recommend", "satisfied", "enjoyed"
    ]
}

# Lowercase all keywords for reliable matching
for k in theme_keywords:
    theme_keywords[k] = list(dict.fromkeys([kw.lower() for kw in theme_keywords[k]]))

# Show the theme list summary
print('Loaded', len(theme_keywords), 'themes. Example theme keys:')
list(theme_keywords.keys())[:6]

In [ ]:
# Helper functions for text normalization and theme matching

def normalize_text(text):
    """Lowercase and normalize whitespace/punctuation for simpler matching."""
    if not isinstance(text, str):
        return ""
    txt = text.lower()
    # Normalize whitespace
    txt = re.sub(r"\s+", " ", txt).strip()
    return txt


def find_themes(text, keywords_map, require_hits=1):
    """Return list of themes that match text.

    - require_hits: minimum number of keyword hits required per theme to consider it matched.
      (Default 1 = any single keyword triggers the theme.)
    """
    txt = normalize_text(text)
    if txt == "":
        return []

    theme_matches = []
    for theme, keywords in keywords_map.items():
        hit_count = 0
        # Prioritize multi-word phrases by checking longer keywords first
        for kw in sorted(keywords, key=lambda x: -len(x)):
            # Word-boundary pattern for single-word keywords; phrase substring allowed
            if ' ' in kw:
                if kw in txt:
                    hit_count += 1
            else:
                # match whole word
                if re.search(r'\b' + re.escape(kw) + r'\b', txt):
                    hit_count += 1
            if hit_count >= require_hits:
                theme_matches.append(theme)
                break
    return theme_matches

# Simple test
print(find_themes('I was exhausted after 12-hour shifts and had back pain from lifting boxes', theme_keywords))
print(find_themes('No onboarding or training; we had to train each other', theme_keywords))

In [ ]:
# Parameters
# set require_hits to 1 for permissive matching, increase to 2 for higher precision
REQUIRE_HITS = 1

# Columns we will process for Glassdoor reviews
glassdoor_summary_col = 'cleaned_summary'
glassdoor_pros_col = 'cleaned_reviewPros'
glassdoor_cons_col = 'cleaned_reviewCons'

# Ensure columns exist
for c in [glassdoor_summary_col, glassdoor_pros_col, glassdoor_cons_col, 'source']:
    if c not in df.columns:
        df[c] = ''

# Identify glassdoor rows (case-insensitive)
is_glass = df['source'].astype(str).str.lower() == 'glassdoor'
print('Glassdoor rows:', is_glass.sum())

# Apply theme matching per field
for col, out_col in [(glassdoor_summary_col, 'categorized_summary'),
                     (glassdoor_pros_col, 'categorized_reviewPros'),
                     (glassdoor_cons_col, 'categorized_reviewCons')]:
    df[out_col] = [[] for _ in range(len(df))]
    df.loc[is_glass, out_col] = df.loc[is_glass, col].fillna('').apply(lambda t: find_themes(t, theme_keywords, REQUIRE_HITS))

# Combined per-row (union preserving order)
def union_lists(a, b, c):
    s = []
    for lst in (a, b, c):
        if isinstance(lst, list):
            s.extend(lst)
    return list(dict.fromkeys(s))

# Create combined only for glassdoor rows
combined_col = 'categorized_combined'
df[combined_col] = [[] for _ in range(len(df))]
df.loc[is_glass, combined_col] = df.loc[is_glass].apply(lambda r: union_lists(r['categorized_summary'], r['categorized_reviewPros'], r['categorized_reviewCons']), axis=1)

print('Sample categorized (first 5 glassdoor rows):')
df.loc[is_glass, ['cleaned_summary','cleaned_reviewPros','cleaned_reviewCons','categorized_combined']].head(5)


In [ ]:
# Process YouTube transcripts (transcript_cleaned)
transcript_col = 'transcript_cleaned'
if transcript_col not in df.columns:
    df[transcript_col] = ''

is_yt = df['source'].astype(str).str.lower() == 'youtube'
print('YouTube rows:', is_yt.sum())

# Create categorized_transcript column
df['categorized_transcript'] = [[] for _ in range(len(df))]
df.loc[is_yt, 'categorized_transcript'] = df.loc[is_yt, transcript_col].fillna('').apply(lambda t: find_themes(t, theme_keywords, REQUIRE_HITS))

# Update categorized_combined for YouTube (union existing combined + transcript themes)
for idx, row in df.loc[is_yt].iterrows():
    existing = row.get('categorized_combined', [])
    if isinstance(existing, str):
        # try to parse as list-like string
        try:
            existing = eval(existing)
            if not isinstance(existing, list):
                existing = []
        except Exception:
            existing = []
    transcript_themes = row.get('categorized_transcript', []) or []
    union = list(dict.fromkeys((existing if isinstance(existing, list) else []) + (transcript_themes if isinstance(transcript_themes, list) else [])))
    df.at[idx, 'categorized_combined'] = union

# Show a few YouTube examples
df.loc[is_yt, [transcript_col, 'categorized_transcript','categorized_combined']].head(6)


In [ ]:
# Create one-hot columns for top themes (useful for aggregation)
# Find top N themes across all 'categorized_combined' entries
all_themes = Counter()
for cell in df['categorized_combined']:
    try:
        items = cell if isinstance(cell, list) else eval(cell) if isinstance(cell, str) and cell.startswith('[') else (cell if isinstance(cell, list) else [])
    except Exception:
        items = []
    for it in items:
        all_themes[it] += 1

TOP_N = 12
top_themes = [t for t,c in all_themes.most_common(TOP_N)]
print('Top themes:', top_themes)

# Add one-hot columns
for theme in top_themes:
    col_name = 'theme__' + re.sub(r'[^0-9a-zA-Z]+', '_', theme.lower()).strip('_')
    def has_theme(cell, theme=theme):
        try:
            items = cell if isinstance(cell, list) else eval(cell) if isinstance(cell, str) and cell.startswith('[') else []
        except Exception:
            items = []
        return int(theme in items)
    df[col_name] = df['categorized_combined'].apply(has_theme)

# Quick aggregation table
agg = df[[c for c in df.columns if c.startswith('theme__')]].sum().sort_values(ascending=False)
agg.head(12)


In [ ]:
# Save the annotated CSV
output_path = '/mnt/data/glassdoor_and_youtube_sentiment_with_full_categories.csv'
df.to_csv(output_path, index=False)
print('Saved annotated file to', output_path)

# Show top theme counts and a sample of annotated rows
agg_display = agg.reset_index()
agg_display.columns = ['theme_col','count']
agg_display.head(12)

# Show sample rows for inspection (glassdoor and youtube)
sample_glass = df[df['source'].astype(str).str.lower() == 'glassdoor'].head(6)
sample_yt = df[df['source'].astype(str).str.lower() == 'youtube'].head(6)

print('\nSample Glassdoor rows with categories:')
print(sample_glass[['cleaned_summary','cleaned_reviewPros','cleaned_reviewCons','categorized_combined']].to_string(index=False))

print('\nSample YouTube rows with transcript categories:')
print(sample_yt[['transcript_cleaned','categorized_transcript','categorized_combined']].to_string(index=False))
